Code for San Francisco

# CTA Homelessness data

## Purpose

The purpose of this notebook is to load in the sample data. Each sheet from the Excel file was saved out to a CSV with the corresponding name.

- Author: Matt Mollison
- Date: 2016-10-05


In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
# %load_ext autoreload
# # the "1" means: always reload modules marked with "%aimport"
# %autoreload 1

from __future__ import absolute_import, division, print_function
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os, sys
# from tqdm import tqdm
# import warnings

sns.set_context("poster", font_scale=1.3)

In [2]:
pd.set_option('display.max_columns', 100)

In [3]:
# Example for adding path
# write code in ../source and add that to runtime path
# src_dir = os.path.join(os.getcwd(), os.pardir, '../source')
# sys.path.append(src_dir)

# Import like so
# # import my_module from the source directory
# %aimport my_module
# from my_module.build_features import remove_invalid_data

In [4]:
# where the data is stored
datadir = os.path.join(os.getenv('HOME'), 'data/DATA_CTA/SampleDataSet-MOSBE&SCz-2012-1001--2016-0531/USE THIS ONE/')

In [6]:
sheet = 'Client'

cols = [
    'Personal ID',
    'Race',
    'Ethnicity',
    'Gender',
    'Veteran Status',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_client = pd.read_csv(infile, header=0, index_col=0, usecols=cols)

df_client = df_client.dropna(how='all')
df_client.index = df_client.index.astype('int')

# drop people that we don't have demographic information for
df_client = df_client.dropna(how='any', subset=['Race', 'Ethnicity', 'Gender'])

# fill in missing values
df_client['Veteran Status'] = df_client['Veteran Status'].fillna(value='')

# Remove "(HUD) from strings
cols = ['Race', 'Ethnicity', 'Veteran Status']
for col in cols:
    df_client[col] = df_client[col].apply(lambda x: x.replace(' (HUD)', ''))

# put the nans back
df_client.loc[df_client['Veteran Status'] == '', 'Veteran Status'] = np.nan

# and encode booleans
col = 'Veteran Status'
df_client.loc[df_client[col] == 'Yes', col] = True
df_client.loc[df_client[col] == 'No', col] = False
df_client.loc[df_client[col] == 'Not Applicable - Child', col] = False
df_client.loc[df_client[col] == 'Client refused', col] = np.nan
df_client.loc[df_client[col] == "Client doesn't know", col] = np.nan
df_client.loc[df_client[col] == 'Data not collected', col] = np.nan

In [7]:
df_client.head()

,Race,Ethnicity,Gender,Veteran Status
Personal ID,,,,
173781,White,Hispanic/Latino,Female,False
173782,White,Hispanic/Latino,Male,False
173783,White,Hispanic/Latino,Female,False
173800,Black or African American,Non-Hispanic/Non-Latino,Female,False
173803,White,Hispanic/Latino,Female,False


In [8]:
sheet = 'Enrollment'

cols = [
    'Personal ID',
    'Project Entry ID',
    'Client Age at Entry',
    'Last Permanent Zip',
    'Entry Date',
    'Exit Date',
    'Project ID',
    'Housing Status @ Project Start',
    'Living situation before program entry?',
    'Client Location',
    'Household ID',
    'Relationship to HoH',
    'Disabling Condition',
    'Continuously Homeless One Year',
    'Times Homeless Past Three Years',
    'Months Homeless This Time',
    'Chronic Homeless',
    'In Permanent Housing',
    'Residential Move In Date',
    'Domestic Violence Victim',
    'DV When Occurred',
    'DV Currently Fleeing',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_enroll = pd.read_csv(infile, header=0, index_col=0, usecols=cols,
                        parse_dates=['Entry Date', 'Exit Date', 'Residential Move In Date'],
                        infer_datetime_format=True)

df_enroll = df_enroll.dropna(axis=0, how='all')
df_enroll.index = df_enroll.index.astype('int')

# drop anyone for whom we don't have age
df_enroll = df_enroll.dropna(subset=['Client Age at Entry'])

# turn these into integers
cols = ['Project Entry ID', 'Client Age at Entry', 'Project ID', 'Household ID']
for col in cols:
    df_enroll[col] = df_enroll[col].astype('int')

# Remove "(HUD) from strings
cols = ['Housing Status @ Project Start',
        'Living situation before program entry?',
        'Disabling Condition',
        'Continuously Homeless One Year',
        'Domestic Violence Victim',
        'DV When Occurred',
        'DV Currently Fleeing',
        ]
for col in cols:
    df_enroll[col] = df_enroll[col].fillna(value='')
    df_enroll[col] = df_enroll[col].apply(lambda x: x.replace(' (HUD)', ''))
    # put the nans back
    df_enroll.loc[df_enroll[col] == '', col] = np.nan

# encode booleans

# cols = [
#     'Disabling Condition',
#     'Continuously Homeless One Year',
#     'Chronic Homeless',
#     'In Permanent Housing',
#     'Domestic Violence Victim',
#     'DV Currently Fleeing', ]

col = 'Disabling Condition'
df_enroll.loc[df_enroll[col] == 'Yes', col] = True
df_enroll.loc[df_enroll[col] == 'No', col] = False
df_enroll.loc[df_enroll[col] == 'Not Applicable - Child', col] = False
df_enroll.loc[df_enroll[col] == 'Client refused', col] = np.nan
df_enroll.loc[df_enroll[col] == "Client doesn't know", col] = np.nan
df_enroll.loc[df_enroll[col] == 'Data not collected', col] = np.nan

col = 'Continuously Homeless One Year'
df_enroll.loc[df_enroll[col] == 'Yes', col] = True
df_enroll.loc[df_enroll[col] == 'No', col] = False
df_enroll.loc[df_enroll[col] == 'Client refused', col] = np.nan
df_enroll.loc[df_enroll[col] == "Client doesn't know", col] = np.nan
df_enroll.loc[df_enroll[col] == 'Data not collected', col] = np.nan

col = 'Chronic Homeless'
df_enroll.loc[df_enroll[col] == 'Yes', col] = True
df_enroll.loc[df_enroll[col] == 'No', col] = False

col = 'In Permanent Housing'
df_enroll.loc[df_enroll[col] == 'Yes', col] = True
df_enroll.loc[df_enroll[col] == 'No', col] = False

col = 'Domestic Violence Victim'
df_enroll.loc[df_enroll[col] == 'Yes', col] = True
df_enroll.loc[df_enroll[col] == 'No', col] = False
df_enroll.loc[df_enroll[col] == 'Not Applicable - Child', col] = False
df_enroll.loc[df_enroll[col] == 'Client refused', col] = np.nan
df_enroll.loc[df_enroll[col] == "Client doesn't know", col] = np.nan
df_enroll.loc[df_enroll[col] == 'Data not collected', col] = np.nan

col = 'DV Currently Fleeing'
df_enroll.loc[df_enroll[col] == 'Yes', col] = True
df_enroll.loc[df_enroll[col] == 'No', col] = False
df_enroll.loc[df_enroll[col] == 'Client refused', col] = np.nan
df_enroll.loc[df_enroll[col] == "Client doesn't know", col] = np.nan
df_enroll.loc[df_enroll[col] == 'Data not collected', col] = np.nan


In [9]:
df_enroll.head()

,Project Entry ID,Client Age at Entry,Last Permanent Zip,Entry Date,Exit Date,Project ID,Housing Status @ Project Start,Living situation before program entry?,Client Location,Household ID,Relationship to HoH,Disabling Condition,Continuously Homeless One Year,Times Homeless Past Three Years,Months Homeless This Time,Chronic Homeless,In Permanent Housing,Residential Move In Date,Domestic Violence Victim,DV When Occurred,DV Currently Fleeing
Personal ID,,,,,,,,,,,,,,,,,,,,,
173781,252608,34,93907,2014-02-28,2014-02-28,2104,At-risk of homelessness,"Rental by client, no ongoing housing subsidy",NaN,252608,NaN,NaN,NaN,NaN,NaN,False,NaN,NaT,True,More than a year ago,NaN
173781,314080,35,93907,2014-05-06,2014-09-30,2101,Category 1 - Homeless,"Emergency shelter, including hotel or motel pa...",CA-506,314080,Head of household's child,NaN,False,2,NaN,False,NaN,NaT,True,From six to twelve months ago,NaN
173782,122915,1,NaN,2004-11-29,NaT,2114,NaN,NaN,CA-506,122914,NaN,NaN,NaN,NaN,NaN,False,NaN,NaT,NaN,NaN,NaN
173782,314084,10,93905,2014-05-06,2014-09-30,2101,Category 1 - Homeless,"Emergency shelter, including hotel or motel pa...",CA-506,314080,Head of household's child,False,True,2,NaN,False,NaN,NaT,True,From six to twelve months ago,NaN
173783,122916,2,NaN,2004-11-29,NaT,2114,NaN,NaN,CA-506,122914,NaN,NaN,NaN,NaN,NaN,False,NaN,NaT,NaN,NaN,NaN


In [10]:
sheet = 'Disability'

cols = [
    'Personal ID',
    'Disability Type',
    'Receiving Services For',
    'Disabilities ID',
    'Project Entry ID',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_disability = pd.read_csv(infile, header=0, index_col=0, usecols=cols)

df_disability = df_disability.dropna(axis=0, how='all')
df_disability.index = df_disability.index.astype('int')

# turn these into integers
cols = ['Disabilities ID', 'Project Entry ID']
for col in cols:
    df_disability[col] = df_disability[col].astype('int')

# Remove "(HUD) from strings
cols = ['Disability Type',
        'Receiving Services For',
        ]
for col in cols:
    df_disability[col] = df_disability[col].fillna(value='')
    df_disability[col] = df_disability[col].apply(lambda x: x.replace(' (HUD)', ''))
    # put the nans back
    df_disability.loc[df_disability[col] == '', col] = np.nan

# encode booleans
col = 'Receiving Services For'
df_disability.loc[df_disability[col] == 'Yes', col] = True
df_disability.loc[df_disability[col] == 'No', col] = False
df_disability.loc[df_disability[col] == 'Client refused', col] = np.nan
df_disability.loc[df_disability[col] == "Client doesn't know", col] = np.nan
df_disability.loc[df_disability[col] == 'Data not collected', col] = np.nan

In [11]:
df_disability.head()

,Disability Type,Receiving Services For,Disabilities ID,Project Entry ID
Personal ID,,,,
173848,Physical,True,971050,382788
173848,Physical,True,971050,383070
173848,Physical,True,971050,383336
173848,Physical,True,971050,383701
173848,Physical,True,971050,383839


In [12]:
sheet = 'HealthInsurance'

cols = [
    'Personal ID',
    'Health Insurance Information Date',
    'Health Insurance',
    'Data Collection Stage',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_healthins = pd.read_csv(infile, header=0, index_col=0, usecols=cols,
                           parse_dates=['Health Insurance Information Date'],
                           infer_datetime_format=True)

df_healthins = df_healthins.dropna(axis=0, how='all')
df_healthins.index = df_healthins.index.astype('int')


In [13]:
df_healthins.head()

,Health Insurance Information Date,Health Insurance,Data Collection Stage
Personal ID,,,
173781,2014-05-06,MEDICAID,Project Entry
173782,2014-05-06,MEDICARE,Project Entry
173783,2014-05-06,MEDICAID,Project Entry
173848,2014-08-22,MEDICAID,Project Entry
173848,2014-08-22,State Health Insurance for Adults,Project Entry


In [14]:
sheet = 'Benefit'

cols = [
    'Personal ID',
    'Non-Cash Benefit',
    'Data Collection Stage',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_benefit = pd.read_csv(infile, header=0, index_col=0, usecols=cols)

df_benefit = df_benefit.dropna(axis=0, how='all')
df_benefit.index = df_benefit.index.astype('int')

# Drop any project missing the code
df_benefit = df_benefit.dropna(how='any', subset=['Non-Cash Benefit'])

# Remove "(HUD) from strings
cols = ['Non-Cash Benefit',
        ]
for col in cols:
    df_benefit[col] = df_benefit[col].fillna(value='')
    df_benefit[col] = df_benefit[col].apply(lambda x: x.replace(' (HUD)', ''))
    # put the nans back
    df_benefit.loc[df_benefit[col] == '', col] = np.nan


In [15]:
df_benefit.head()

,Non-Cash Benefit,Data Collection Stage
Personal ID,,
173781,Supplemental Nutrition Assistance Program (Foo...,Project Entry
173781,Supplemental Nutrition Assistance Program (Foo...,Project Entry
173803,Supplemental Nutrition Assistance Program (Foo...,Project Entry
173848,Supplemental Nutrition Assistance Program (Foo...,Project Entry
173848,Supplemental Nutrition Assistance Program (Foo...,Project Entry


In [16]:
# TODO: turn dollar strings into integers, need to account for commas

sheet = 'Income Entry & Exit'

cols = [
    'Personal ID',
    'Project Entry ID',
    'Entry Alimony',
    'Entry Child Support',
    'Entry Earned',
    'Entry GA',
    'Entry Other',
    'Entry Pension',
    'Entry Private Disability',
    'Entry Social Security Retirement',
    'Entry SSDI',
    'Entry SSI',
    'Entry TANF',
    'Entry Unemployment',
    'Entry VA Non-Service',
    'Entry VA Service Connected',
    "Entry Worker's Compensation",
    'Entry Total Income',
    'Exit Alimony',
    'Exit Child Support',
    'Exit Earned',
    'Exit GA',
    'Exit Other',
    'Exit Pension',
    'Exit Private Disability',
    'Exit Social Security Retirement',
    'Exit SSDI',
    'Exit SSI',
    'Exit TANF',
    'Exit Unemployment',
    'Exit VA Non-Service',
    'Exit VA Service Connected',
    "Exit Worker's Compensation",
    'Exit Total Income',
    'Income Change',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_income = pd.read_csv(infile, header=0, index_col=0, usecols=cols)

df_income = df_income.dropna(axis=0, how='all')
df_income.index = df_income.index.astype('int')

# turn these into integers
cols = ['Project Entry ID']
for col in cols:
    df_income[col] = df_income[col].astype('int')

# assume all nans are $0
df_income = df_income.fillna(value='0')

# turn the dollar strings into integers
for col in df_income.columns:
    if col != 'Project Entry ID':
        df_income[col] = df_income[col].str.replace(',', '')
        df_income[col] = df_income[col].str.replace(r'[^-+\d.]', '').astype(int)

In [18]:
df_income.head()

,Project Entry ID,Entry Alimony,Entry Child Support,Entry Earned,Entry GA,Entry Other,Entry Pension,Entry Private Disability,Entry Social Security Retirement,Entry SSDI,Entry SSI,Entry TANF,Entry Unemployment,Entry VA Non-Service,Entry VA Service Connected,Entry Worker's Compensation,Entry Total Income,Exit Alimony,Exit Child Support,Exit Earned,Exit GA,Exit Other,Exit Pension,Exit Private Disability,Exit Social Security Retirement,Exit SSDI,Exit SSI,Exit TANF,Exit Unemployment,Exit VA Non-Service,Exit VA Service Connected,Exit Worker's Compensation,Exit Total Income,Income Change
Personal ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
173781,252608,0,0,0,0,0,0,0,0,0,0,607,0,0,0,0,607,0,0,0,0,0,0,0,0,0,0,607,0,0,0,0,607,0
173781,314080,0,0,0,0,0,0,0,0,0,0,1143,0,0,0,0,1143,0,0,0,0,0,0,0,0,0,0,1143,0,0,0,0,1143,0
173782,122915,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
173782,314084,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
173783,122916,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
sheet = 'Service'

cols =  [
    'Personal ID',
    'Services ID',
    'Date Provided',
    'Date Ended',
    'Service Code',
    'Description',
    'Project ID',
    'Record Type',
    'Project Entry ID',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_service = pd.read_csv(infile, header=0, index_col=0, usecols=cols,
                         parse_dates=['Date Provided', 'Date Ended'],
                         infer_datetime_format=True)

df_service = df_service.dropna(axis=0, how='all')
df_service.index = df_service.index.astype('int')

# Drop anyone missing these IDs
df_service = df_service.dropna(how='any', subset=['Project ID', 'Project Entry ID'])

# turn these into integers
cols = ['Project ID', 'Project Entry ID']
for col in cols:
    df_service[col] = df_service[col].astype('int')


In [20]:
df_service.head()

,Services ID,Date Provided,Date Ended,Service Code,Description,Project ID,Record Type,Project Entry ID
Personal ID,,,,,,,,
175155,7312345,2013-04-01,2013-04-02,BH-1800,Emergency Shelter,2840,Bed Night,382789
180665,7312348,2013-04-01,2013-04-02,BH-1800,Emergency Shelter,2840,Bed Night,382791
184317,7312343,2013-04-01,2013-04-02,BH-1800,Emergency Shelter,2840,Bed Night,382790
184431,7312350,2013-04-01,2013-04-02,BH-1800,Emergency Shelter,2840,Bed Night,483580
184502,7312340,2013-04-01,2013-04-02,BH-1800,Emergency Shelter,2840,Bed Night,382785


In [21]:
sheet = 'Project'

cols = [
    'Project ID',
    'Project Name',
    'Project Type Code',
    'Address City',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_project = pd.read_csv(infile, header=0, index_col=1, usecols=cols)

df_project.head()

df_project = df_project.dropna(axis=0, how='all')
df_project.index = df_project.index.astype('int')

# Drop any project missing the code
df_project = df_project.dropna(how='any', subset=['Project Type Code'])

# Remove "(HUD) from strings
cols = ['Project Type Code',
        ]
for col in cols:
    df_project[col] = df_project[col].fillna(value='')
    df_project[col] = df_project[col].apply(lambda x: x.replace(' (HUD)', ''))
    # put the nans back
    df_project.loc[df_project[col] == '', col] = np.nan


In [22]:
df_project.head()

,Project Name,Project Type Code,Address City
Project ID,,,
2142,MOSBE CHS - Elm House,Transitional housing,NaN
2142,MOSBE CHS - Elm House,Transitional housing,NaN
3417,MOSBE CHS - RHY - BCP ES,Emergency Shelter,Monterey
3417,MOSBE CHS - RHY - BCP ES,Emergency Shelter,Seaside
3418,MOSBE CHS - RHY - BCP - HP,Homelessness Prevention,Monterey


In [23]:
sheet = 'BedInventory'

cols = [
    'Project ID',
    'Inventory ID',
    'Inventory Household Type',
    'HMIS Participating Beds',
    'Inventory Start Date',
    'Inventory End Date',
    'Unit Inventory',
    'Bed Inventory',
    'Vet Bed Inventory',
    'Youth Bed Inventory',
    'Youth Bed Age Group',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_bedinv = pd.read_csv(infile, header=0, index_col=0, usecols=cols,
                        parse_dates=['Inventory Start Date', 'Inventory End Date'],
                        infer_datetime_format=True)

df_bedinv = df_bedinv.dropna(axis=0, how='all')
df_bedinv.index = df_bedinv.index.astype('int')

# turn these into integers, assume zero if NaN
cols = ['Inventory ID', 'HMIS Participating Beds', 'Unit Inventory', 'Bed Inventory', 'Vet Bed Inventory', 'Youth Bed Inventory']
for col in cols:
    # df_bedinv[col] = df_bedinv.loc[~df_bedinv[col].isnull(), col].apply(lambda x: int(x))
    df_bedinv[col] = df_bedinv[col].fillna(value=0)
    df_bedinv[col] = df_bedinv[col].astype('int')


In [24]:
df_bedinv.head()

,Inventory ID,Inventory Household Type,HMIS Participating Beds,Inventory Start Date,Inventory End Date,Unit Inventory,Bed Inventory,Vet Bed Inventory,Youth Bed Inventory,Youth Bed Age Group
Project ID,,,,,,,,,,
2088,144,Households with at least one adult and one child,36,2001-01-01,2013-09-30,9,36,0,0,NaN
2088,655,Households with at least one adult and one child,36,2013-10-01,NaT,9,36,0,0,NaN
2095,194,Households without children,0,2004-01-01,2013-09-30,2,2,0,2,Only Ages 18-24
2095,678,Households without children,0,2013-10-01,NaT,4,4,0,4,Only Under Age 18
2101,145,Households with at least one adult and one child,24,1992-01-01,2013-09-30,5,24,0,0,NaN
